### Setup

In [ ]:
import numpy as np

from hot_dust import preprocess, model

In [ ]:
from importlib import reload

preprocess = reload(preprocess)
model = reload(model)

In [ ]:
ds = preprocess.prepare_training_data()
train, validate, test = preprocess.to_tensorflow(preprocess.split_training_data(ds))

### Parameters

In [ ]:
epochs = 5 # Max 500
batch_size = 64
buffer_size = 10 * batch_size

model = reload(model)  # DEBUG

train = train.shuffle(buffer_size).batch(batch_size)
validate = validate.batch(batch_size)

layer = model.pretraining(dataset=test)

In [ ]:
network = model.compile(normalization=layer)
network.summary()

### Fitting

In [ ]:
import tensorflow as tf 
tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

fit = network.fit(
    train,
    epochs=epochs,
    validation_data=validate,
    verbose=2,
)

network.save("data/network")
fit = {"epoch": fit.epoch, **fit.history}
np.savez("data/fit.npz", fit)

### Training and Validation Losses

In [ ]:
model.plot_loss(fit)